In [ ]:
from PIL import Image
import numpy as np
from google.colab import files
import cv2

def calculate_distance(ball_image_1, ball_image_2, known_distance, ball_diameter_cm=20):
    # Convert the images to grayscale
    ball_1_gray = np.array(ball_image_1.convert("L"))
    ball_2_gray = np.array(ball_image_2.convert("L"))

    # Step 1: Apply Gaussian blur to reduce noise before edge detection
    ball_1_blur = cv2.GaussianBlur(ball_1_gray, (5, 5), 0)
    ball_2_blur = cv2.GaussianBlur(ball_2_gray, (5, 5), 0)

    # Step 2: Use edge detection (Sobel) to detect the ball's edges
    sobel_1 = cv2.Sobel(ball_1_blur, cv2.CV_64F, 1, 0, ksize=5)
    sobel_2 = cv2.Sobel(ball_2_blur, cv2.CV_64F, 1, 0, ksize=5)

    # Convert the Sobel result to uint8 (8-bit format) for thresholding
    sobel_1_8bit = np.uint8(np.abs(sobel_1))
    sobel_2_8bit = np.uint8(np.abs(sobel_2))

    # Step 3: Apply dynamic threshold (Otsu's method)
    _, ball_1_edges = cv2.threshold(sobel_1_8bit, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, ball_2_edges = cv2.threshold(sobel_2_8bit, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Step 4: Find contours to detect the ball's boundary
    contours_1, _ = cv2.findContours(ball_1_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_2, _ = cv2.findContours(ball_2_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Ensure that at least one contour is found in each image
    if len(contours_1) == 0 or len(contours_2) == 0:
        raise ValueError("Ball contour not detected in one of the images.")

    # Find the bounding boxes around the largest contour (assuming it's the ball)
    ball_1_contour = max(contours_1, key=cv2.contourArea)
    ball_2_contour = max(contours_2, key=cv2.contourArea)

    x1, y1, w1, h1 = cv2.boundingRect(ball_1_contour)
    x2, y2, w2, h2 = cv2.boundingRect(ball_2_contour)

    # Use the width of the bounding box as the ball's width in pixels
    ball_1_width_px = w1
    ball_2_width_px = w2

    # Step 5: Calculate the distance using the inverse proportion
    distance_ball_2_meters = known_distance * (ball_1_width_px / ball_2_width_px)

    return distance_ball_2_meters

# Example usage:
uploaded = files.upload()  # Upload your two images
ball_image_1 = Image.open(list(uploaded.keys())[0])  # First image (ball_1)
ball_image_2 = Image.open(list(uploaded.keys())[1])  # Second image (ball_2)
known_distance = 1.0  # Distance of the ball in the first image (in meters)

# Calculate the distance
distance = calculate_distance(ball_image_1, ball_image_2, known_distance)
print(f"The distance of the ball in the second image is approximately {distance:.2f} meters.")




Saving Picture1.jpg to Picture1 (4).jpg
Saving Picture2.jpg to Picture2 (4).jpg
The distance of the ball in the second image is approximately 1.00 meters.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y scikit-image
!pip install scikit-image==0.18.3

Found existing installation: scikit-image 0.24.0
Uninstalling scikit-image-0.24.0:
  Successfully uninstalled scikit-image-0.24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 34.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 105.9 MB/s eta 0:00:00
  Created wheel for scikit-image: filename=scikit_image-0.18.3-cp310-cp310-linux_x86_64.whl size=36001484 sha256=d5d472144fe425fda6eb71c142dd2840ee36c9f2d44049adb9c364c099e38959
  Stored in directory: /root/.cache/pip/wheels/9d/55/80/c5bf730bc10906b1518cd6ebc05c87e69d1c46099ebdc9f59f
Successfully built scikit-image
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.15 requires scikit-image>=0.21.0, but you have scikit-image 0.18.

In [ ]:
import numpy as np
from skimage.feature import greycomatrix, greycoprops
from skimage.measure import shannon_entropy

# Matrix extracted from the uploaded image
image = np.array([
    [2, 2, 2, 2, 2, 1, 1, 1, 3, 3, 3, 6, 3, 6, 3],
    [3, 3, 3, 3, 2, 2, 2, 1, 3, 3, 3, 6, 7, 6, 3],
    [4, 4, 5, 5, 4, 3, 1, 1, 1, 2, 2, 1, 2, 1, 1],
    [4, 5, 5, 5, 4, 3, 1, 2, 1, 2, 3, 2, 2, 2, 1],
    [4, 4, 5, 5, 4, 3, 1, 2, 1, 2, 3, 4, 4, 3, 2],
    [4, 4, 5, 5, 4, 3, 1, 1, 1, 2, 3, 4, 4, 4, 3],
    [4, 4, 4, 3, 3, 2, 1, 2, 1, 2, 3, 4, 4, 4, 3],
    [4, 4, 4, 3, 3, 2, 1, 2, 2, 3, 4, 4, 4, 4, 3],
    [1, 1, 1, 2, 3, 3, 3, 2, 2, 3, 4, 4, 3, 3, 3],
    [3, 4, 4, 4, 3, 2, 1, 2, 1, 2, 3, 4, 4, 4, 3],
    [3, 4, 4, 4, 3, 2, 1, 2, 1, 2, 3, 4, 4, 4, 3],
    [3, 3, 3, 3, 3, 2, 1, 2, 1, 2, 3, 4, 4, 4, 3],
    [2, 3, 4, 4, 4, 2, 1, 2, 1, 2, 2, 2, 3, 3, 3],
    [2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
])

# Define the number of levels based on the maximum value in the image
levels = np.max(image) + 1  # Adjusting levels to include the maximum pixel value

# GLCM calculation at distance 1 and angle 0
glcm = greycomatrix(image, distances=[1], angles=[0], levels=levels, symmetric=True, normed=True)

# Calculating GLCM properties
contrast = greycoprops(glcm, 'contrast')[0, 0]
homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
energy = greycoprops(glcm, 'energy')[0, 0]

# Entropy calculation
entropy = shannon_entropy(image)

# Return the results
print(f'Contrast: {contrast:.4f}')
print(f'Homogeneity: {homogeneity:.4f}')
print(f'Energy: {energy:.4f}')
print(f'Entropy: {entropy:.4f}')



Contrast: 0.8571
Homogeneity: 0.7143
Energy: 0.2910
Entropy: 2.2804
